# **Trying out a new way of loading time series data**





# The score might fluctuate from 0.483 to 0.487 due to some randomness 

In [2]:
import numpy as np
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis
import numpy as np
import polars as pl
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from sklearn.base import clone
from copy import deepcopy
import optuna
from scipy.optimize import minimize
import os
import matplotlib.pyplot as plt
import seaborn as sns

import re
from colorama import Fore, Style

from tqdm import tqdm
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import *
from sklearn.metrics import *

SEED = 42
n_splits = 5

import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None


def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)

    # Convert 'time_of_day' to datetime
    df['time_of_day'] = pd.to_datetime(df['time_of_day'])
    df.set_index('time_of_day', inplace=True)

    # Daily aggregation
    daily_features = df.resample('D').agg({
        'X': ['mean', 'std', 'max', 'min', 'median', 'skew', lambda x: kurtosis(x, nan_policy='omit')],
        'Y': ['mean', 'std', 'max', 'min', 'median', 'skew', lambda x: kurtosis(x, nan_policy='omit')],
        'Z': ['mean', 'std', 'max', 'min', 'median', 'skew', lambda x: kurtosis(x, nan_policy='omit')],
        'enmo': ['mean', 'std', 'max', 'min', 'median', 'skew', lambda x: kurtosis(x, nan_policy='omit')],
        'light': ['mean', 'std'],
        'battery_voltage': ['mean', 'std'],
        'non-wear_flag': 'sum'
    })

    # Flatten the MultiIndex columns
    daily_features.columns = ['_'.join(col).strip() for col in daily_features.columns.values]
    
    # Add additional features
    daily_features['activity_count'] = (df['enmo'] > 0.1).resample('D').sum()  # Adjust threshold as needed
    daily_features['days_active'] = (df['non-wear_flag'] == 0).resample('D').sum()  # Count active days
    
    # Rate of change features
    daily_features['enmo_change'] = daily_features['enmo_mean'].diff()
    
    # Rolling statistics
    rolling_windows = [5, 10, 15]  # Days
    for window in rolling_windows:
        daily_features[f'enmo_rolling_mean_{window}'] = daily_features['enmo_mean'].rolling(window=window).mean()
        daily_features[f'enmo_rolling_std_{window}'] = daily_features['enmo_std'].rolling(window=window).std()

    # Frequency domain features using FFT
    for axis in ['X', 'Y', 'Z', 'enmo']:
        freq_features = np.fft.fft(df[axis])
        daily_features[f'{axis}_dominant_freq'] = np.abs(freq_features).argmax()

    # Reset index to retain 'id'
    daily_features.reset_index(inplace=True)
    
    # Extract child ID from filename
    child_id = filename.split('=')[1]
    daily_features['id'] = child_id

    return daily_features



def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    # Concatenate all results into a single DataFrame
    features_df = pd.concat(results, ignore_index=True)

    return features_df

# Build the autoencoder model
def build_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    
    # Encoder: compressing the input
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    
    # Decoder: reconstructing the input
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    
    # Autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    
    # Encoder model (for getting the compressed representation)
    encoder = Model(inputs=input_layer, outputs=encoded)
    
    autoencoder.compile(optimizer=Adam(), loss='mse')
    
    return autoencoder, encoder

# Function to perform dimensionality reduction using an autoencoder
def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    """
    Perform dimensionality reduction using an Autoencoder.

    Parameters:
    df (pd.DataFrame): The input DataFrame with numerical features.
    encoding_dim (int): The dimension of the encoded space.
    epochs (int): Number of epochs to train the autoencoder.
    batch_size (int): Size of the batches for training.

    Returns:
    pd.DataFrame: DataFrame containing the reduced-dimensional representation (encoding).
    """
    
    # Step 1: Standardize the data
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    # Step 2: Build the autoencoder
    input_dim = df_scaled.shape[1]
    autoencoder, encoder = build_autoencoder(input_dim, encoding_dim)
    
    # Step 3: Train the autoencoder
    autoencoder.fit(df_scaled, df_scaled, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=1)
    
    # Step 4: Get the encoded (reduced) representation
    encoded_data = encoder.predict(df_scaled)
    
    # Step 5: Create a DataFrame for the encoded features
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i+1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

# Load the data
train = pd.read_csv('/Users/ad53533/Desktop/Applied ML/Project/train.csv')
test = pd.read_csv('/Users/ad53533/Desktop/Applied ML/Project/test.csv')
sample = pd.read_csv('/Users/ad53533/Desktop/Applied ML/Project/sample_submission.csv')

train_ts = load_time_series("/Users/ad53533/Desktop/Applied ML/Project/series_train.parquet")
test_ts = load_time_series("/Users/ad53533/Desktop/Applied ML/Project/series_test.parquet")

 16%|█▌        | 157/997 [00:23<02:07,  6.60it/s]


NotADirectoryError: [Errno 20] Not a directory: '/Users/ad53533/Desktop/Applied ML/Project/series_train.parquet/.DS_Store/part-0.parquet'

In [3]:
# Columns to drop
columns_to_drop = [
    'enmo_change',
    'enmo_rolling_mean_5',
    'enmo_rolling_std_5',
    'enmo_rolling_mean_10',
    'enmo_rolling_std_10',
    'enmo_rolling_mean_15',
    'enmo_rolling_std_15'
]

# Drop the columns
train_ts.drop(columns=columns_to_drop, inplace=True)

# Drop the columns
test_ts.drop(columns=columns_to_drop, inplace=True)


KeyError: "['enmo_change', 'enmo_rolling_mean_5', 'enmo_rolling_std_5', 'enmo_rolling_mean_10', 'enmo_rolling_std_10', 'enmo_rolling_mean_15', 'enmo_rolling_std_15'] not found in axis"

In [ ]:
train_ts

In [ ]:
# Drop 'id' column for training
df_train = train_ts.drop('id', axis=1)
df_test = test_ts.drop('id', axis=1)

In [ ]:
df_train.dtypes

In [ ]:
df_train

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

def build_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    
    # Encoder: compressing the input
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    
    # Decoder: reconstructing the input
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    
    # Autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    
    # Encoder model (for getting the compressed representation)
    encoder = Model(inputs=input_layer, outputs=encoded)
    
    autoencoder.compile(optimizer=Adam(), loss='mse')
    
    return autoencoder, encoder

def perform_autoencoder(df, encoding_dim=50, epochs=200, batch_size=32):
    scaler = StandardScaler()
    
    # Exclude non-numeric columns
    df_numeric = df.select_dtypes(include=[np.number])
    
    # Check for and handle NaN values
    if df_numeric.isnull().values.any():
        df_numeric.fillna(df_numeric.mean(), inplace=True)

    if np.isinf(df_numeric).values.any():
        df_numeric.replace([np.inf, -np.inf], np.nan, inplace=True)
        df_numeric.fillna(df_numeric.mean(), inplace=True)

    df_scaled = scaler.fit_transform(df_numeric)

    # Check for NaN values after scaling
    if np.isnan(df_scaled).any() or np.isinf(df_scaled).any():
        print("NaN or infinite values detected in the scaled data!")
        return None  # or handle as needed

    input_dim = df_scaled.shape[1]
    autoencoder, encoder = build_autoencoder(input_dim, encoding_dim)
    
    # Use a smaller learning rate for stability
    autoencoder.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
    
    # Set up early stopping
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Train the autoencoder with early stopping
    autoencoder.fit(df_scaled, df_scaled, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    shuffle=True, 
                    verbose=1, 
                    callbacks=[early_stopping])

    encoded_data = encoder.predict(df_scaled)
    
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i+1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded


In [ ]:
# # Perform autoencoder dimensionality reduction
# train_ts_encoded = perform_autoencoder(df_train, encoding_dim=60, epochs=100, batch_size=32)
# test_ts_encoded = perform_autoencoder(df_test, encoding_dim=60, epochs=100, batch_size=32)

train_ts_encoded = perform_autoencoder(df_train, encoding_dim=64, epochs=256, batch_size=32)
test_ts_encoded = perform_autoencoder(df_test, encoding_dim=64,epochs=256, batch_size=32)


In [ ]:
train_ts_encoded

In [ ]:
test_ts_encoded

In [ ]:
train_ts_encoded

In [ ]:
time_series_cols = train_ts_encoded.columns.tolist()
#time_series_cols.remove("id")

In [ ]:
train_ts_encoded["id"]=train_ts["id"]

test_ts_encoded['id']=test_ts["id"]
#test_ts_pca["id"]=test_ts["id"]

In [ ]:
train_ts_encoded

In [ ]:
#train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts_encoded, how="left", on='id')
#test_ = pd.merge(test, test_ts, how="left", on='id')

train = pd.merge(train, train_ts_encoded, how="left", on='id')
#test = pd.merge(test, test_ts, how="inner", on='id')

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
analysis=train

In [ ]:
analysis = analysis.dropna(subset='Enc_4')

In [ ]:
analysis=analysis.dropna(subset='sii')

In [ ]:
analysis = analysis.drop('id', axis=1)

In [ ]:
def engineer_features(df):
    # Create interaction features
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    
    # Create categorical features
    #df['Age_Group'] = pd.cut(df['Basic_Demos-Age'], bins=[0, 12, 18, 25, 100], labels=['Child', 'Teen', 'Young Adult', 'Adult'])
    #df['BMI_Category'] = pd.cut(df['Physical-BMI'], bins=[0, 18.5, 25, 30, 100], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
    
    return df

test = engineer_features(test)

In [ ]:
test.shape

In [ ]:
train=engineer_features(train)

In [ ]:
train.shape

In [ ]:
#train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [ ]:
nan_rows = train.query('sii.isna()', engine='python')

In [ ]:
# train.isna().sum().sum()-train_imputed.isna().sum().sum()

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer

# Assuming 'train' is your DataFrame

# Step 1: Create the KNN imputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Step 2: Select numeric columns and fit the imputer
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
imputed_data = imputer.fit_transform(train[numeric_cols])

# Step 3: Create a new DataFrame with the imputed values
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)

# Step 4: Convert the 'sii' column back to integers
train_imputed['sii'] = train_imputed['sii'].round().astype(int)

# If there are other columns to retain, you can merge them back
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]

# Now, check if 'sii' has been filled and is of integer type
print(train_imputed['sii'].isna().sum())  # Should be 0 if all NaNs were filled
print(train_imputed['sii'].dtype)  # Should show 'int'

train_imputed['sii']=train['sii']

train=train_imputed

In [ ]:
%%time

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii','BMI_Age','Internet_Hours_Age','BMI_Internet_Hours']

featuresCols += time_series_cols

train = train[featuresCols]
#test= test[featuresCols]
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 
          'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

def update(df):
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

In [ ]:
featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday','BMI_Age','Internet_Hours_Age','BMI_Internet_Hours']

featuresCols += time_series_cols


In [ ]:
test= test[featuresCols]

In [ ]:
# #train['Age_Group'] = train['Age_Group'].cat.add_categories(['Missing'])

# # Step 2: Fill missing values with 'Missing'
# #train['Age_Group'] = train['Age_Group'].fillna('Missing')

#train['BMI_Category'] = train['BMI_Category'].cat.add_categories(['Missing'])

# # Step 2: Fill missing values with 'Missing'
#train['BMI_Category'] = train['BMI_Category'].fillna('Missing')

In [ ]:
# #test['Age_Group'] = test['Age_Group'].cat.add_categories(['Missing'])

# # Step 2: Fill missing values with 'Missing'
# #test['Age_Group'] = test['Age_Group'].fillna('Missing')

#test['BMI_Category'] = test['BMI_Category'].cat.add_categories(['Missing'])

# # Step 2: Fill missing values with 'Missing'
#test['BMI_Category'] = test['BMI_Category'].fillna('Missing')

In [ ]:
test

In [ ]:
# #train = train.drop('BMI_Category', axis=1)
# test = test.drop('BMI_Category', axis=1)

In [ ]:
train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

In [ ]:
"""This Mapping Works Fine For me I also Check Each Values in Train and test Using Logic. There no Data Lekage."""

for col in cat_c:
    mapping_train = create_mapping(col, train)
    mapping_test = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping_train).astype(int)
    test[col] = test[col].replace(mapping_test).astype(int)

print(f'Train Shape : {train.shape} || Test Shape : {test.shape}')

In [ ]:
%%time

train.head()

In [ ]:
%%time

test.head()

In [ ]:
train.sii.unique()

In [ ]:
# Find columns missing in df2 but present in df1
set(train.columns) - set(test.columns)

In [ ]:
analysis=train.dropna(subset='Enc_51')

In [ ]:
missing_df=train[train['Enc_51'].isna()]

In [ ]:
missing_df

In [ ]:
import pandas as pd

# Assuming analysis is your complete DataFrame (1000 data points)
# and missing_df is your DataFrame with missing values (3000 data points)

# Step 1: Store original data types
original_dtypes = analysis.dtypes.to_dict()

# Step 2: Split the complete data into features and target
X_complete = analysis.drop('sii', axis=1)
y_complete = analysis['sii']

# Step 3: Identify categorical and numerical columns
categorical_features = [
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
    'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
    'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season'
]

# Step 3.1: Drop categorical columns for mean calculation
numerical_df = analysis.drop(columns=categorical_features + ['sii'])

# Group by the target variable and calculate means for numerical features
mean_impute = numerical_df.groupby(analysis['sii']).mean()

# Group by the target variable and calculate modes for categorical features
mode_impute = analysis[categorical_features].groupby(analysis['sii']).agg(
    lambda x: x.mode()[0] if not x.mode().empty else None
)

# Step 4: Fill missing values class-wise
def fill_missing_values(row):
    class_value = row['sii']
    if pd.isnull(class_value):
        return row  # No action if class_value is NaN

    for col in row.index:
        if pd.isnull(row[col]):
            if col in mean_impute.columns and class_value in mean_impute.index:  # For numerical columns
                row[col] = mean_impute.loc[class_value, col]
            elif col in mode_impute.columns and class_value in mode_impute.index:  # For categorical columns
                row[col] = mode_impute.loc[class_value, col]
    return row

# Apply the filling function to each row in the missing_df
df_missing_filled = missing_df.apply(fill_missing_values, axis=1)

# Step 5: Combine datasets
final_dataset = pd.concat([analysis, df_missing_filled], ignore_index=True)

# Step 6: Convert categorical features to int64 safely
for col in categorical_features:
    # Check for unique values and their counts
    unique_values = final_dataset[col].unique()
    print(f'Unique values in {col}: {unique_values}')

    # Fill NaNs with a placeholder (optional)
    final_dataset[col] = final_dataset[col].fillna('missing_value')

    # Convert to category type first to manage the encoding
    final_dataset[col] = final_dataset[col].astype('category')
    
    # Factorize the column
    final_dataset[col] = pd.factorize(final_dataset[col])[0].astype('int64')

# Step 7: Restore original data types for other columns
final_dataset = final_dataset.astype(original_dtypes)

# Now, final_dataset contains all your data with missing values filled and original data types restored


In [ ]:
mean_impute.columns

In [ ]:
mode_impute.columns

In [ ]:
train=final_dataset

In [ ]:
missing_df['FGC-Season']

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize
from tqdm import tqdm
from IPython.display import clear_output

# Function definitions (unchanged)
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    # Ensure all categorical columns are encoded
    for col in categorical_features:
        X[col] = pd.factorize(X[col])[0].astype('int64')
    
    test_data = test_data.copy()
    for col in categorical_features:
        test_data[col] = pd.factorize(test_data[col])[0].astype('int64')

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {tKappa:.3f}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

# Model parameters and instantiation (unchanged)
Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,
    'lambda_l2': 0.01
}

XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': SEED
}

CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'cat_features': cat_c,
    'verbose': 0,
    'l2_leaf_reg': 10
}

Light = LGBMRegressor(**Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Combine models using Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

# Train the ensemble model
Submission = TrainML(voting_model, test)

# Save submission
# Submission.to_csv('submission.csv', index=False)


In [ ]:
test.shape

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

SEED = 42
n_splits = 5

# Load datasets
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')




def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]



def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

        
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)   


##########

# import pandas as pd
# from sklearn.impute import KNNImputer

# # Assuming 'train' is your DataFrame

# # Step 1: Create the KNN imputer
# imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# # Step 2: Select numeric columns and fit the imputer
# numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
# imputed_data = imputer.fit_transform(train[numeric_cols])

# # Step 3: Create a new DataFrame with the imputed values
# train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)

# # Step 4: Convert the 'sii' column back to integers
# train_imputed['sii'] = train_imputed['sii'].round().astype(int)

# # If there are other columns to retain, you can merge them back
# for col in train.columns:
#     if col not in numeric_cols:
#         train_imputed[col] = train[col]

# # Now, check if 'sii' has been filled and is of integer type
# print(train_imputed['sii'].isna().sum())  # Should be 0 if all NaNs were filled
# print(train_imputed['sii'].dtype)  # Should show 'int'

# train_imputed['sii']=train['sii']

# train=train_imputed


#########


featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

featuresCols += time_series_cols

train = train[featuresCols]
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
          'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
          'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df
        
train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

# Model parameters for LightGBM
Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,  # Increased from 6.59
    'lambda_l2': 0.01  # Increased from 2.68e-06
}


# XGBoost parameters
XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,  # Increased from 0.1
    'reg_lambda': 5,  # Increased from 1
    'random_state': SEED
}


CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'cat_features': cat_c,
    'verbose': 0,
    'l2_leaf_reg': 10  # Increase this value
}

# Create model instances
Light = LGBMRegressor(**Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Combine models using Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

# Train the ensemble model
Submission1 = TrainML(voting_model, test)

# Save submission
#Submission1.to_csv('submission.csv', index=False)


In [ ]:
Submission1

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

SEED = 42
n_splits = 5

# Load datasets
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    stats, indexes = zip(*results)
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)


#####


# import pandas as pd
# from sklearn.impute import KNNImputer

# # Assuming 'train' is your DataFrame

# # Step 1: Create the KNN imputer
# imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# # Step 2: Select numeric columns and fit the imputer
# numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
# imputed_data = imputer.fit_transform(train[numeric_cols])

# # Step 3: Create a new DataFrame with the imputed values
# train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)

# # Step 4: Convert the 'sii' column back to integers
# train_imputed['sii'] = train_imputed['sii'].round().astype(int)

# # If there are other columns to retain, you can merge them back
# for col in train.columns:
#     if col not in numeric_cols:
#         train_imputed[col] = train[col]

# # Now, check if 'sii' has been filled and is of integer type
# print(train_imputed['sii'].isna().sum())  # Should be 0 if all NaNs were filled
# print(train_imputed['sii'].dtype)  # Should show 'int'

# train_imputed['sii']=train['sii']

# train=train_imputed

#####

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-Season', 'CGAS-CGAS_Score',
                'Physical-Season', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'Fitness_Endurance-Season',
                'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec', 'FGC-Season',
                'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
                'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone',
                'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat',
                'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_A-Season',
                'PAQ_A-PAQ_A_Total', 'PAQ_C-Season', 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
                'PreInt_EduHx-Season', 'PreInt_EduHx-computerinternet_hoursday', 'sii']

featuresCols += time_series_cols

train = train[featuresCols]
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
         'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tp_rounded = threshold_Rounder(tpm, KappaOPtimizer.x)

    return tp_rounded

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Imputation step: Filling missing values with the median
imputer = SimpleImputer(strategy='median')

# Updating the ensemble to include the RandomForest and GradientBoosting models
ensemble = VotingRegressor(estimators=[
    ('lgb', Pipeline(steps=[('imputer', imputer), ('regressor', LGBMRegressor(random_state=SEED))])),
    ('xgb', Pipeline(steps=[('imputer', imputer), ('regressor', XGBRegressor(random_state=SEED))])),
    ('cat', Pipeline(steps=[('imputer', imputer), ('regressor', CatBoostRegressor(random_state=SEED, silent=True))])),
    ('rf', Pipeline(steps=[('imputer', imputer), ('regressor', RandomForestRegressor(random_state=SEED))])),
    ('gb', Pipeline(steps=[('imputer', imputer), ('regressor', GradientBoostingRegressor(random_state=SEED))]))
])

# Train the ensemble with the updated model pipeline
predictions = TrainML(ensemble, test)

# Save predictions to a CSV file
sample['sii'] = predictions
#sample.to_csv('submission.csv', index=False)

In [ ]:
sample

In [ ]:
import pandas as pd

# Load your three submission files
sub1 = Submission
sub2 = Submission1
sub3 = sample

# Ensure the IDs are aligned (if not sorted)
sub1 = sub1.sort_values(by='id').reset_index(drop=True)
sub2 = sub2.sort_values(by='id').reset_index(drop=True)
sub3 = sub3.sort_values(by='id').reset_index(drop=True)

# Combine the three predictions
combined = pd.DataFrame({
    'id': sub1['id'],
    'sii_1': sub1['sii'],
    'sii_2': sub2['sii'],
    'sii_3': sub3['sii']
})

# Apply majority voting
def majority_vote(row):
    return row.mode()[0]  # Mode gets the most frequent value (majority vote)

combined['final_sii'] = combined[['sii_1', 'sii_2', 'sii_3']].apply(majority_vote, axis=1)

# Create the final submission DataFrame
final_submission = combined[['id', 'final_sii']].rename(columns={'final_sii': 'sii'})

# Save the final submission to a CSV file
final_submission.to_csv('submission.csv', index=False)

print("Majority voting completed and saved to 'Final_Submission.csv'")

In [ ]:
# # Load your three submission files
# sub1 = Submission
# sub2 = Submission1
# sub3 = sample

# sub1 = sub1.sort_values(by='id').reset_index(drop=True)
# sub2 = sub2.sort_values(by='id').reset_index(drop=True)
# sub3 = sub3.sort_values(by='id').reset_index(drop=True)

# sub1 = sub1.rename(columns={'sii': 'sii_1'})
# sub2 = sub2.rename(columns={'sii': 'sii_2'})
# sub3 = sub3.rename(columns={'sii': 'sii_3'})
# subs = pd.merge(sub1,sub2,on=['id'])
# subs = pd.merge(subs,sub3,on=['id'])

# #subs['sii_s'] = subs['sii_1'] *0.555 + 0.001* subs['sii_2'] + 0.444* subs['sii_3']
# #subs['sii_s'] = subs['sii_1'] *0.450 + 0.450* subs['sii_2'] + 0.100* subs['sii_3']
# #subs['sii_s'] = subs['sii_1'] *0.525 + 0.050* subs['sii_2'] + 0.425* subs['sii_3']
# #subs['sii_s'] = subs['sii_1'] *0.500 + 0.100* subs['sii_2'] + 0.400* subs['sii_3']

# subs['sii_s'] = np.round(subs['sii_1'] *0.85 + 0.10* subs['sii_2'] + 0.05* subs['sii_3'])

# subs['sii_s'] = subs['sii_s'].astype(int)

# combined = pd.DataFrame({
    
#     'id'   : sub1['id'],
    
#     'sii_1': sub1['sii_1'],
#     'sii_2': sub2['sii_2'],
#     'sii_3': sub3['sii_3'],
    
#     'sii_s': subs['sii_s'],
# })

# display(combined)

# def majority_vote(row):
#     return row.mode()[0]
                                                         
# combined['final_sii'] = combined[['sii_1', 'sii_2', 'sii_3', 'sii_s']].apply(majority_vote, axis=1)

# final_submission = combined[['id', 'final_sii']].rename(columns={'final_sii': 'sii'})

# final_submission.to_csv('submission.csv', index=False)

# print("Majority voting completed and saved to 'Final_Submission.csv'")

In [ ]:
# X = train.drop(['sii'], axis=1)
# y = train['sii']

In [ ]:
# X_train=X
# y_train=y

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import VotingRegressor
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor
# from catboost import CatBoostRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# # Define hyperparameter grids for each model

# # LightGBM hyperparameter grid
# lgb_param_grid = {
#     'regressor__learning_rate': [0.01, 0.03, 0.046, 0.1],
#     'regressor__max_depth': [8, 12, 16],
#     'regressor__num_leaves': [128, 256, 478, 512],
#     'regressor__min_data_in_leaf': [10, 13, 16],
#     'regressor__feature_fraction': [0.7, 0.8, 0.893],
#     'regressor__bagging_fraction': [0.7, 0.784, 0.85],
#     'regressor__lambda_l1': [5, 10, 15],
#     'regressor__lambda_l2': [0.001, 0.01, 0.1]
# }

# # XGBoost hyperparameter grid
# xgb_param_grid = {
#     'regressor__learning_rate': [0.01, 0.05, 0.1],
#     'regressor__max_depth': [4, 6, 8],
#     'regressor__n_estimators': [100, 200, 300],
#     'regressor__subsample': [0.7, 0.8, 0.9],
#     'regressor__colsample_bytree': [0.7, 0.8, 0.9],
#     'regressor__reg_alpha': [0.1, 1, 5],
#     'regressor__reg_lambda': [1, 5, 10]
# }

# # CatBoost hyperparameter grid
# cat_param_grid = {
#     'regressor__learning_rate': [0.03, 0.05, 0.07],
#     'regressor__depth': [4, 6, 8],
#     'regressor__iterations': [200, 300, 400],
#     'regressor__l2_leaf_reg': [3, 10, 15]
# }

# # RandomForest hyperparameter grid
# rf_param_grid = {
#     'regressor__n_estimators': [100, 200, 300],
#     'regressor__max_depth': [10, 20, 30],
#     'regressor__min_samples_split': [2, 5, 10],
#     'regressor__min_samples_leaf': [1, 2, 4]
# }

# # GradientBoosting hyperparameter grid
# gb_param_grid = {
#     'regressor__learning_rate': [0.01, 0.05, 0.1],
#     'regressor__n_estimators': [100, 200, 300],
#     'regressor__max_depth': [3, 5, 7],
#     'regressor__subsample': [0.7, 0.8, 0.9]
# }

# # Imputer for handling missing values
# imputer = SimpleImputer(strategy='median')

# # Pipelines for each regressor
# lgb_pipeline = Pipeline(steps=[('imputer', imputer), ('regressor', LGBMRegressor(random_state=SEED))])
# xgb_pipeline = Pipeline(steps=[('imputer', imputer), ('regressor', XGBRegressor(random_state=SEED))])
# cat_pipeline = Pipeline(steps=[('imputer', imputer), ('regressor', CatBoostRegressor(random_state=SEED, silent=True))])
# rf_pipeline = Pipeline(steps=[('imputer', imputer), ('regressor', RandomForestRegressor(random_state=SEED))])
# gb_pipeline = Pipeline(steps=[('imputer', imputer), ('regressor', GradientBoostingRegressor(random_state=SEED))])

# # Perform RandomizedSearchCV for each model

# # LightGBM
# lgb_search = RandomizedSearchCV(lgb_pipeline, lgb_param_grid, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=SEED)
# lgb_search.fit(X_train, y_train)

# # XGBoost
# xgb_search = RandomizedSearchCV(xgb_pipeline, xgb_param_grid, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=SEED)
# xgb_search.fit(X_train, y_train)

# # CatBoost
# cat_search = RandomizedSearchCV(cat_pipeline, cat_param_grid, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=SEED)
# cat_search.fit(X_train, y_train)

# # RandomForest
# rf_search = RandomizedSearchCV(rf_pipeline, rf_param_grid, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=SEED)
# rf_search.fit(X_train, y_train)

# # GradientBoosting
# gb_search = RandomizedSearchCV(gb_pipeline, gb_param_grid, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=SEED)
# gb_search.fit(X_train, y_train)

# # Get the best models from the RandomizedSearchCV results
# best_lgb = lgb_search.best_estimator_
# best_xgb = xgb_search.best_estimator_
# best_cat = cat_search.best_estimator_
# best_rf = rf_search.best_estimator_
# best_gb = gb_search.best_estimator_

# # Create a new ensemble with the best estimators
# ensemble = VotingRegressor(estimators=[
#     ('lgb', best_lgb),
#     ('xgb', best_xgb),
#     ('cat', best_cat),
#     ('rf', best_rf),
#     ('gb', best_gb)
# ])

# # Train the ensemble
# predictions = TrainML(ensemble, test)

# # Save predictions to a CSV file
# sample['sii'] = predictions
# sample.to_csv('submission.csv', index=False)